Section 1: 라이브러리 설치 및 불러오기

In [ ]:
# Hugging Face 모델용 필수 라이브러리 설치
!pip install -q transformers accelerate

# 기본 라이브러리 import
import os
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
import transformers
import warnings

# HuggingFace 모델 import
from transformers import (
    Blip2Processor, Blip2ForConditionalGeneration,
    CLIPTokenizer, CLIPTextModel
)


In [ ]:
# tqdm 진행률 바 깨지지 않도록 설정
transformers.logging.set_verbosity_error()  # transformers 내부 경고 숨김
warnings.filterwarnings("ignore")           # 일반 파이썬 경고도 숨김


Section 2: 경로 및 프롬프트 정의

In [ ]:
# 학습 이미지 디렉토리 (class 1~7 폴더 구조)
train_dir = "/content/drive/MyDrive/VLM/train_half"

# BLIP2에 넣을 프롬프트
prompt = "Question: What emotion is the person showing in the image? Answer:"


Section 3: 모델 로딩 (BLIP2 + CLIP)

In [ ]:
# BLIP2 모델: 이미지 → 감정 설명 문장 생성
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
blip_model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-flan-t5-xl",
    device_map="auto",  # 자동으로 GPU 할당
    torch_dtype=torch.float16
).eval()

# CLIP 텍스트 인코더: 문장 → 벡터
clip_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
clip_text_model = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").to("cuda").eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Section 4: BLIP2 → 감정 설명 문장 생성 + CLIP → 텍스트 feature 추출

In [ ]:
# 결과 저장 리스트
clip_text_features = []
img_names = []

# Class 1~7 순회
for emotion in range(1, 8):
    folder = os.path.join(train_dir, str(emotion))
    image_files = [f for f in os.listdir(folder) if f.endswith((".jpg", ".png"))]

    for i, f in enumerate(tqdm(image_files, desc=f"Class {emotion}")):
        path = os.path.join(folder, f)

        try:
            # 이미지 열기 및 전처리
            with Image.open(path) as img:
                img = img.convert("RGB").resize((224, 224))

                # BLIP2로 문장 생성
                blip_inputs = processor(images=img, text=prompt, return_tensors="pt").to("cuda", torch.float16)
                with torch.no_grad():
                    generated_ids = blip_model.generate(
                        **blip_inputs,
                        max_new_tokens=30  # 불필요한 파라미터 제거
                    )
                    text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

                # CLIP으로 문장 인코딩 (텍스트 feature 추출)
                clip_inputs = clip_tokenizer([text], return_tensors="pt").to("cuda")
                with torch.no_grad():
                    text_feature = clip_text_model(**clip_inputs).pooler_output  # (1, D)

                # feature와 이미지 이름 저장
                clip_text_features.append(text_feature.squeeze().cpu().numpy())
                img_names.append(f)

        except Exception as e:
            tqdm.write(f"에러: {path} → {e}")


Class 7: 100%|██████████| 1009/1009 [45:42<00:00,  2.72s/it]


Section 5: npy, csv 결과 저장

In [ ]:
import numpy as np
import pandas as pd

# 저장 경로 설정
save_dir = "/content/drive/MyDrive/VLM"
os.makedirs(save_dir, exist_ok=True)

#image_name + feature dict로 묶기
merged_data = [
    {"image_name": name, "clip_feature": feat}
    for name, feat in zip(img_names, clip_text_features)
]

#NPY 저장 (하나의 파일로)
npy_path = os.path.join(save_dir, "train_CLIP_feature.npy")
np.save(npy_path, merged_data)

#CSV 저장 (string 형태로 변환)
csv_path = os.path.join(save_dir, "train_CLIP_feature.csv")
df = pd.DataFrame({
    "image_name": [item["image_name"] for item in merged_data],
    "clip_feature": [",".join(map(str, item["clip_feature"])) for item in merged_data]
})
df.to_csv(csv_path, index=False)

print(f"저장 완료: \n- NPY: {npy_path}\n- CSV: {csv_path}")


저장 완료: 
- NPY: /content/drive/MyDrive/VLM/train_CLIP_feature.npy
- CSV: /content/drive/MyDrive/VLM/train_CLIP_feature.csv


In [ ]:
loaded = np.load("/content/drive/MyDrive/VLM/train_CLIP_feature.npy", allow_pickle=True)
print(loaded[0]["image_name"])        # 예: 'train_00123.jpg'
print(loaded[0]["clip_feature"][:5])  # 예: [-0.23, 0.87, -1.02, ...]


train_07148_aligned.jpg
[-1.5942181  -0.6751456   1.4848552  -0.00326018 -0.83672017]


In [ ]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("/content/drive/MyDrive/VLM/train_CLIP_feature.csv")

# 첫 번째 clip_feature 문자열 → 리스트로 변환
first_feat = df["clip_feature"].iloc[0]
feat_vector = list(map(float, first_feat.split(",")))

# 길이 확인
print(f"feature vector 차원 수: {len(feat_vector)}")


feature vector 차원 수: 768
